In [2]:
from vpython import *
import math
import numpy as np
from sympy import Eq, Symbol, solve

scene = canvas(background = color.white)

def createRay(start, angle, size=2):
    v1 = vec(0,-1,0)
    return arrow(pos = start, axis = size * v1.rotate(radians(angle)),
                 shaftwidth = 0.2, headwidth = 0.3, headlength = 0.2, color = color.yellow)
def refractRay(refIndex1, refIndex2, vector_inc):
    v1 = vec(0,-1,0)
    theta_inc = diff_angle(v1,vector_inc.axis)
    theta_ref = asin((refIndex1/refIndex2) * sin(theta_inc))
    return arrow(pos = vector_inc.pos+vector_inc.axis, axis = v1.rotate(theta_ref),
                 shaftwidth = 0.2, headwidth = 0.3, headlength = 0.2, color = color.yellow)

# Accepts User Input for the Refractive Profile
i = input("n(y) = ")
y = Symbol('y')
medium = i

if i == "ceil(y)":
    print("Match Found")
    lowerBound = 99
    upperBound = 159
    print("Bounds: [",lowerBound, ",",upperBound,"]")
   
elif "y" not in i:
    print("Constant Function Not Accepted")
    i = input("n(y) = ")
    y = Symbol('y')
    medium = i
   
else:
    print("Match Found")

    lowerProfile = Eq(eval(medium), 1)
    upperProfile = Eq(eval(medium), 1.6)
    bounds = [abs(round(float(solve(lowerProfile)[0]),3)),
              abs(round(float(solve(upperProfile)[0]),3))]
    lowerBound = np.amin(bounds)
    upperBound = np.amax(bounds)
   
    print("Bounds: [",lowerBound, ",",upperBound,"]")
   
resolution = 50
   
def refractiveProfile(y):
    if i == "ceil(y)":
        return math.ceil(y)/100
    elif "y" not in i:
        return int(i)
    else:
        return eval(i, {'y': y, 'np': np})

def d_fun(x):
    h = 1e-5
    return (refractiveProfile(x+h)-refractiveProfile(x-h))/(2*h)

currentResolution = 50

x = np.linspace(lowerBound+0.01,upperBound-0.01, currentResolution)
indices = []

for k in x:
    indices.append(refractiveProfile(k))

change = d_fun(np.random.uniform(lowerBound,upperBound))
if change < 0:
    print("Function is decreasing. Order must be reversed. [::-1]")
    indicesFinal = (np.round(np.sort(np.asarray(indices)),2)[::-1])[1:-1]
else:
    print("Function is increasing. Preserve order.")
    indicesFinal = (np.round(np.sort(np.asarray(indices)),2))[1:-1]

light = createRay(vec(0,0,0), 35)
v = [light]

maximumResolution = 0
if i == "ceil(y)":
    maximumResolution = 47
else:
    for i in range(0, len(indicesFinal)-2):
        if (indicesFinal[i+2] == indicesFinal[i]):
            maximumResolution = i
if maximumResolution == 0:
    maximumResolution = currentResolution
           
print("The maximum resolution is ", maximumResolution)

for i in range (0, maximumResolution-3):
    t=0
    while t < 1:  
        rate(5)
        sphere()
        newLight = refractRay(indicesFinal[i], indicesFinal[i+1], v[i])
        v.append(newLight)
       
        # Updates center of screen to be in the middle of the rays
        horSum = 0
        verSum = 0
        for i in range(0,len(v)):
            horSum += (v[i].pos.x+(v[i].axis.x)/2)
            verSum += (v[i].pos.y+(v[i].axis.y/2))
        horAvg = horSum / len(v)
        verAvg = verSum / len(v)
        hi = vec(horAvg,verAvg,0)
        scene.center = hi
        t+=1

<IPython.core.display.Javascript object>

n(y) = y**(-1/2)
Match Found
Bounds: [ 0.391 , 1.0 ]
Function is decreasing. Order must be reversed. [::-1]
The maximum resolution is  50
